In [1]:
!pip3 install minio
!pip3 install pandas
!sudo apt-get install -y mosquitto-clients

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mosquitto-clients is already the newest version (1.6.9-1).
0 upgraded, 0 newly installed, 0 to remove and 74 not upgraded.


In [2]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

#df_metrics = pd.DataFrame(columns=['Workload_size','Workload TPT','Average Workload TPT','Average FET','Average CT'])
http://172.17.141.197:3000/goto/MU1mA-MVk?orgId=1

In [3]:
#scenario="emqx_testing_namespace_scaling_without_max_inflight_replicas_2_10_3"
scenario = "testing_resource_limits"
user=1
iteration=1
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:9096/'


In [4]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv


In [25]:
!locust --headless -f mqtt_locust.py --host='127.0.0.1' -t 3  --users 1 --spawn-rate 1

[2022-09-15 08:05:30,070] ubuntu/WARNING/locust.main: System open file limit '4096' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2022-09-15 08:05:30,071] ubuntu/INFO/locust.main: Run time limit set to 3 seconds
[2022-09-15 08:05:30,071] ubuntu/INFO/locust.main: Starting Locust 2.12.0
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2022-09-15 08:05:30,075] ubuntu/INFO/locust.runners: Ramping to 1 users at a rate of 1.00 per second
[2022-09-15 08:05:30,076] ubuntu/INFO/locust.runners: 

In [6]:
#cmd = "./mqtt_client.sh "+str(user)
#with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
#    output1 = process.communicate()[0].decode("utf-8")
time.sleep(4)

In [7]:

df= pd.read_csv('input_data.csv')
print(df)

                         intime
0  '2022-09-15 08:00:35.032841'


In [8]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)


In [9]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime']).dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'])
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2022-09-15 08:00:35.032841,2022-09-15 08:00:40.272


0 days 00:00:05.239159


In [10]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))    


In [11]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-ggwl6  -n openfaas -c gateway  | grep /function/scaling-aeneas-mqtt | cut -c 93-97 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-ggwl6  -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]


In [12]:
#print(df['aeneas'])
#print(df['tocloud'])
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime']).dt.seconds
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
df['Waiting time']= (df['TPT']).astype(float) - df['FET']

display(df)
df.to_csv("data/"+str(user)+"_"+str(iteration)+"_"+scenario+".csv")

,intime,outtime,aeneas,tocloud,TPT,FET,Waiting time
0,2022-09-15 08:00:35.032841,2022-09-15 08:00:40.272,4.62,0.0406,5,4.6606,0.3394


In [13]:
user=df.shape[0]
metrics = {'workload_type':str(user)+"_"+scenario,'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean(),'Average Waiting Time':df['Waiting time'].mean()}


In [14]:
pre_url = PROMETHEUS + '/api/v1/query?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time = (time.mktime(timestamp.timetuple()))
#interval= int(df['TPT'].sum()) if int(df['TPT'].sum()) > 60  else 60


In [15]:

interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [16]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

60


In [17]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    data=res.get('data').get('result')[0].get('value')[1]
    return data

In [18]:
expr_cpu='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace=~"openfaas.*|default"}[5m]))&time='+str(time)
expr_memory='sum(container_memory_working_set_bytes{container_name!="POD",namespace=~"openfaas.*|default"})&time='+str(time)
expr_disk_read ='sum(rate(container_fs_reads_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
expr_disk_write='sum(rate(container_fs_writes_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
expr_network_transmit='sum(rate(container_network_transmit_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
expr_network_recieve='sum(rate(container_network_receive_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
#expr_cpu = '100 - (avg (irate(node_cpu_seconds_total{mode="idle"}['+str(interval)+'s])) * 100)&time='+str(time)
#expr_memory = '100 - ((sum(node_memory_MemAvailable_bytes) / sum(node_memory_MemTotal_bytes))*100)&time='+str(time)
#expr_disk_read = 'sum(rate(node_disk_read_bytes_total['+str(interval)+'s]))&time='+str(time)
#expr_disk_write =  'sum(rate(node_disk_written_bytes_total['+str(interval)+'s]))&time='+str(time)
#expr_network_transmit= 'sum(rate(node_network_transmit_errs_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
#expr_network_recieve='sum(rate(node_network_receive_packets_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
metrics['C_CPU'] = getdataprometheus(pre_url+expr_cpu)
metrics['C_RAM'] = getdataprometheus(pre_url+expr_memory)
metrics['C_disk_read']=getdataprometheus(pre_url+expr_disk_read)
metrics['C_disk_write']=getdataprometheus(pre_url+expr_disk_write)
metrics['C_network_recieve']=getdataprometheus(pre_url+expr_network_transmit)
metrics['C_network_transmit']=getdataprometheus(pre_url+expr_network_recieve)

In [19]:
#Pod Stats
expr_pod_cpu_aeneas_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_broker = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"emqx.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"tocloud.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_functions_all = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time)

print(expr_pod_cpu_aeneas_function)
metrics['pod_cpu_aeneas_function']=getdataprometheus(pre_url+expr_pod_cpu_aeneas_function)
metrics['pod_cpu_mosquitto_broker']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_broker)
metrics['pod_cpu_mosquitto_connector_aeneas']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_aeneas)
metrics['pod_cpu_mosquitto_connector_tocloud']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_tocloud)
metrics['pod_cpu_functions_all']=getdataprometheus(pre_url+expr_pod_cpu_functions_all)



sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}[60s]))&time=1663228840.0


In [20]:
df_metrics = pd.read_csv("data/locus_emqx_data.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)


/tmp/ipykernel_164525/1187223197.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(metrics, ignore_index=True)


In [21]:
df_metrics.to_csv("data/locus_emqx_data.csv",index=False)
#df_metrics=df_metrics.sort_values(by="workload_size")
display(df_metrics)

,workload_type,workload_size,Workload TPT,Average Workload TPT,Average FET,Average Waiting Time,C_CPU,C_RAM,C_disk_read,C_disk_write,C_network_recieve,C_network_transmit,pod_cpu_aeneas_function,pod_cpu_functions_all,pod_cpu_mosquitto_broker,pod_cpu_mosquitto_connector_aeneas,pod_cpu_mosquitto_connector_tocloud
0,13_emqx_testing_namespace_scaling_without_max_...,13,65.749086,10.000000,4.774877,5.225123,0.963413,3846922240,2621.413824,51003.281885,572740.216331,641379.317591,2.174775,2.203279,0.040815,0.435942,0.033479
1,25_emqx_testing_namespace_scaling_without_max_...,25,127.036330,8.360000,4.763956,3.596044,1.463696,3884310528,4394.053992,66074.603123,570035.532676,643071.747345,1.806399,1.834895,0.036246,0.52893,0.049043
2,59_emqx_testing_namespace_scaling_without_max_...,59,304.790162,8.016949,4.752527,3.264422,2.907422,3978952704,4110.968799,94579.257989,583303.895307,666981.816433,1.809514,1.839391,0.051075,0.436225,0.047914
3,1_testing_resource_limits,1,5.239159,5.000000,4.660600,0.339400,0.5346384565068855,3674284032,2226.6516870220707,75847.98659410665,26381.693216400137,27682.98179105505,0.004145908457648263,0.009657569575452702,0.03630116215920318,0.06549864136396143,0.03427493673359128


In [22]:
pod_metrics = {}
expr_pod_cpu_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['cpu_aeneas'] = getdataprometheus(pre_url+expr_pod_cpu_aeneas)
expr_pod_memory_aeneas = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['memory_aeneas'] = getdataprometheus(pre_url+expr_pod_memory_aeneas)
expr_pod_cpu_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['cpu_tocloud'] = getdataprometheus(pre_url+expr_pod_cpu_aeneas)
expr_pod_memory_tocloud = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-tolcoud.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['memory_tocloud'] = getdataprometheus(pre_url+expr_pod_memory_aeneas)


In [24]:
#pod_metrics = json.loads(pod_metrics)
df2 = pd.DataFrame.from_dict(pod_metrics, orient="index")
print(df2)

                                   0
cpu_aeneas      0.004145908457648263
memory_aeneas                      0
cpu_tocloud     0.004145908457648263
memory_tocloud                     0
